In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [6]:
df_brd_w_prd_v1_drop_lg_3220 = pd.read_csv("find_missing_df_brd_w_prd_v1_drop_lg_3220.csv")
df_brd_w_prd_v1_drop_lg_3220 = df_brd_w_prd_v1_drop_lg_3220.iloc[:, 1:]

In [11]:
df_brd_w_prd_v1_drop_lg_3220.loc[df_brd_w_prd_v1_drop_lg_3220["crawling_prd_cnt"] < df_brd_w_prd_v1_drop_lg_3220["prd_len"], :]

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt,prd_len
32,https://www.glowpick.com/brands/7406,고려은단,"['https://www.glowpick.com/products/97645', 'h...","['비타민C 1000', '비타민C 골드 플러스', '비타민C 1000 이지', '...",https://www.glowpick.com/brands/7406,"['이너뷰티', '마스크/팩']",17.0,18.0
152,https://www.glowpick.com/brands/11349,글로우,"['https://www.glowpick.com/products/142634', '...","['식이섬유 핑크 스틱', '콜라겐 퍼플 스틱', '면역 옐로우 스틱', '컬러 푸...",https://www.glowpick.com/brands/11349,['이너뷰티'],1137.0,1140.0
163,https://www.glowpick.com/brands/10911,글린트 바이 비디보브,"['https://www.glowpick.com/products/139568', '...","['하이라이터', '아이팔레트', '글리터 젤', '스파클 립스틱', '쉬어 립스틱...",https://www.glowpick.com/brands/10911,"['페이스메이크업', '립메이크업', '아이메이크업', '컨투어링']",6.0,7.0
208,https://www.glowpick.com/brands/9974,끌,"['https://www.glowpick.com/products/128543', '...","['리프팅 마스크', '시그니처 오 드 뚜왈렛', '끌로에 오 드 퍼퓸', '에빠끌...",https://www.glowpick.com/brands/9974,['마스크/팩'],446.0,450.0
209,https://www.glowpick.com/brands/6189,끌라삐엘,"['https://www.glowpick.com/products/89101', 'h...","['리치 아이크림', '보건용 황사방역 마스크 KF94 대형', '히비스커스 릴렉싱...",https://www.glowpick.com/brands/6189,"['스킨케어', '페이스메이크업', '립메이크업', '클렌징', '마스크/팩', '...",18.0,22.0
...,...,...,...,...,...,...,...,...
3172,https://www.glowpick.com/brands/1486,빠이요,"['https://www.glowpick.com/products/73787', 'h...","['https://www.glowpick.com/products/73787', 'h...",https://www.glowpick.com/brands/1486,"['스킨케어', '립메이크업', '클렌징', '마스크/팩', '선케어', '배쓰&바디']",20.0,21.0
3190,https://www.glowpick.com/brands/683,삐아,"['https://www.glowpick.com/products/89222', 'h...","['라스트 블러쉬 [쉐딩]', '섀이드 앤 섀도우', '라스트 벨벳 립 틴트5', ...",https://www.glowpick.com/brands/683,"['스킨케어', '페이스메이크업', '립메이크업', '아이메이크업', '컨투어링',...",121.0,123.0
3725,https://www.glowpick.com/brands/133,스킨푸드,"['https://www.glowpick.com/products/137691', '...","['https://www.glowpick.com/products/137691', '...",https://www.glowpick.com/brands/133,"['스킨케어', '페이스메이크업', '립메이크업', '아이메이크업', '컨투어링',...",1146.0,1148.0
3748,https://www.glowpick.com/brands/618,스트라이벡틴,"['https://www.glowpick.com/products/95848', 'h...","['https://www.glowpick.com/products/95848', 'h...",https://www.glowpick.com/brands/618,"['스킨케어', '마스크/팩']",12.0,14.0


In [8]:
df_brd_w_prd_v1_drop_lg_3220.loc[df_brd_w_prd_v1_drop_lg_3220["crawling_prd_cnt"] > df_brd_w_prd_v1_drop_lg_3220["prd_len"], :]

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt,prd_len
104,https://www.glowpick.com/brands/933,그램,"['https://www.glowpick.com/products/24009', 'h...","['더 촉촉한 크림 50그램', '더 화사한 크림 50', '더블케미 반반팩 2.0...",https://www.glowpick.com/brands/933,"['스킨케어', '마스크/팩']",233.0,140.0
526,https://www.glowpick.com/brands/840,다이소,"['https://www.glowpick.com/products/34534', 'h...","['고급 파우더 브러시 (대)', '블렌딩총알브러시', '핑크팝 브러시', '5단 ...",https://www.glowpick.com/brands/840,"['스킨케어', '페이스메이크업', '립메이크업', '아이메이크업', '컨투어링',...",348.0,347.0
771,https://www.glowpick.com/brands/10935,더마브,"['https://www.glowpick.com/products/139748', '...","['에이 패스트 세럼', '타임와이즈 마이크로더마브레이존 스텝2', 'W389 더마...",https://www.glowpick.com/brands/10935,['스킨케어'],15.0,14.0
1348,https://www.glowpick.com/brands/52,러쉬,"['https://www.glowpick.com/products/41439', 'h...","['더 컴포터', '더 올리브 브랜치', '섹스 밤', '로즈 잼', '브라이트사이...",https://www.glowpick.com/brands/52,"['스킨케어', '페이스메이크업', '립메이크업', '남성화장품', '아이메이크업'...",4574.0,3780.0
1501,https://www.glowpick.com/brands/10861,로아,"['https://www.glowpick.com/products/139124', '...","['콜라겐 젤리 미스트', '프로 아이 팔레트 AD', '프로 아이 팔레트', '프...",https://www.glowpick.com/brands/10861,['스킨케어'],124.0,123.0
1510,https://www.glowpick.com/brands/11977,로에,"['https://www.glowpick.com/products/148840', '...","['라일락 스카이 오 드 뚜왈렛', '화이트셔츠 오 드 뚜왈렛', '하쉬 그린 오 ...",https://www.glowpick.com/brands/11977,['프래그런스'],1071.0,1070.0
1512,https://www.glowpick.com/brands/9855,로에베,"['https://www.glowpick.com/products/126942', '...","['로에베 001 우먼 오 드 뚜왈렛', '로에베 001 맨 오 드 뚜왈렛', '아...",https://www.glowpick.com/brands/9855,['프래그런스'],270.0,269.0
1529,https://www.glowpick.com/brands/9888,로제드 자르딘,"['https://www.glowpick.com/products/127667', '...","['보석 페이스 롤러', '로즈쿼츠 엔젤윙 괄사 마사지도구']",https://www.glowpick.com/brands/9888,['마스크/팩'],2.0,0.0
2056,https://www.glowpick.com/brands/1032,메르필,"['https://www.glowpick.com/products/79741', 'h...","['AC+ 매직클렌져', '멀티 뉴트리션 부스팅 토너', '멀티 뉴트리션 에센스',...",https://www.glowpick.com/brands/1032,"['스킨케어', '클렌징', '마스크/팩', '배쓰&바디']",14.0,13.0


In [12]:
df_v4_dup = pd.read_csv("df_v4_dup.csv")
df_v4_dup = df_v4_dup.iloc[:, 1:]

In [492]:
df_v4_dup

,level_0,index,브랜드 link,브랜드명,브랜드 id,제품 link,제품 id,제품명,제품 용량,제품 가격,...,제품 성분구성: 낮은 위험도 (성분(설명)),제품 성분구성: 중간 위험도 (갯수),제품 성분구성: 중간 위험도 (성분(한글)),제품 성분구성: 중간 위험도(성분(영어)),제품 성분구성: 중간 위험도 (성분(설명)),제품 성분구성: 높은 위험도 (갯수),제품 성분구성: 높은 위험도 (성분(한글)),제품 성분구성: 높은 위험도 (성분(영어)),제품 성분구성: 높은 위험도 (성분(설명)),제품 카테고리


In [487]:
df_v4_dup.loc[df_v4_dup["제품 id"] == 124256, :]

,level_0,index,브랜드 link,브랜드명,브랜드 id,제품 link,제품 id,제품명,제품 용량,제품 가격,...,제품 성분구성: 낮은 위험도 (성분(설명)),제품 성분구성: 중간 위험도 (갯수),제품 성분구성: 중간 위험도 (성분(한글)),제품 성분구성: 중간 위험도(성분(영어)),제품 성분구성: 중간 위험도 (성분(설명)),제품 성분구성: 높은 위험도 (갯수),제품 성분구성: 높은 위험도 (성분(한글)),제품 성분구성: 높은 위험도 (성분(영어)),제품 성분구성: 높은 위험도 (성분(설명)),제품 카테고리
75,1650,1650,NaN,솔더랩,NaN,https://www.glowpick.com/products/124256,124256,[단종] 클렌징 워터,250ml,"18,000원",...,[],0개,[],[],[],0개,[],[],[],"['클렌징', '메이크업클렌저', '클렌징워터']"
151,64210,64250,,솔더랩,NaN,https://www.glowpick.com/products/124256,124256,클렌징 워터,250ml,"18,000원",...,[],0개,[],[],[],0개,[],[],[],"['클렌징', '메이크업클렌저', '클렌징워터']"


In [490]:
df_v4_dup = df_v4_dup.drop(151)

In [482]:
need_to_crawl_lst.append(150332)

In [501]:
for i in range(len(need_to_crawl_lst)):
    need_to_crawl_lst[i] = 'https://www.glowpick.com/products/' + str(need_to_crawl_lst[i])

In [503]:
import pickle

with open("v4_addition_crawling_lst.txt", "wb") as f:
    pickle.dump(need_to_crawl_lst, f)

In [488]:
v4_update = v4_update.drop(64210)

In [493]:
v4_update.to_csv("v4_update.csv")

In [504]:
len(v4_update)

64167

In [507]:
v4_update

,브랜드 link,브랜드명,브랜드 id,제품 link,제품 id,제품명,제품 용량,제품 가격,유사 제품군 랭킹(int),유사 제품군 랭킹(text),...,제품 성분구성: 낮은 위험도 (성분(설명)),제품 성분구성: 중간 위험도 (갯수),제품 성분구성: 중간 위험도 (성분(한글)),제품 성분구성: 중간 위험도(성분(영어)),제품 성분구성: 중간 위험도 (성분(설명)),제품 성분구성: 높은 위험도 (갯수),제품 성분구성: 높은 위험도 (성분(한글)),제품 성분구성: 높은 위험도 (성분(영어)),제품 성분구성: 높은 위험도 (성분(설명)),제품 카테고리
0,NaN,게볼,NaN,https://www.glowpick.com/products/93849,93849,겔라산 핸드크림,75ml,가격미정,577위,핸드크림,...,[],0개,[],[],[],0개,[],[],[],"['배쓰&바디', '핸드케어', '핸드크림']"
1,NaN,게볼,NaN,https://www.glowpick.com/products/36978,36978,메드 데오드란트 풋 크림,125ml,"12,900원",41위,풋크림,...,"['피부컨디셔닝제, 용제', '결합제, 착향제, 피부컨디셔닝제(유연제)', '피부컨...",13개,"['프로필렌글라이콜', '사이클로메티콘', '징크옥사이드', '다이메티콘', '페녹...","['Propylene Glycol', 'Cyclomethicone', 'Zinc O...","['향료, 보습제, 피부보습제, 피부컨디셔닝제, 용제, 점도감소제', '헤어컨디셔닝...",5개,"['프로필파라벤', '부틸파라벤', '아이소부틸파라벤', '쿠마린', '향료']","['Propylparaben', 'Butylparaben', 'Isobutylpar...","['향료, 살균보존제', '향료, 살균보존제', '보존제', '향료', '착향제']","['배쓰&바디', '풋케어', '풋크림']"
2,NaN,구달,NaN,https://www.glowpick.com/products/111421,111421,청귤 비타C 크림 인텐스,50ml,"22,000원",15위,브라이트닝크림,...,"['', '피부컨디셔닝제, 용제', '변성제, 착향제, 헤어컨디셔닝제, 보습제, 피...",5개,"['폴리솔베이트60', '레몬껍질오일', '일랑일랑꽃오일', '폴리솔베이트80', ...","['Polysorbate 60', 'Citrus Limon (Lemon) Peel ...","['향료, 계면활성제-유화제, 계면활성제-용해보조제', '착향제, 피부컨디셔닝제(기...",0개,[],[],[],"['스킨케어', '크림', '브라이트닝크림']"
3,NaN,구달,NaN,https://www.glowpick.com/products/150371,150371,살구 콜라겐 탄력 크림,50ml,"33,000원",90위,안티에이징크림,...,"['피부컨디셔닝제', '변성제, 착향제, 헤어컨디셔닝제, 보습제, 피부보호제, 피부...",2개,"['다이메티콘', '사이클로펜타실록세인']","['Dimethicone', 'Cyclopentasiloxane']","['기포방지제, 피부보호제, 피부컨디셔닝제(수분차단제)', '모발컨디셔닝제, 피부유...",0개,[],[],[],"['스킨케어', '크림', '안티에이징크림']"
4,NaN,구달,NaN,https://www.glowpick.com/products/150374,150374,살구 콜라겐 탄력 앰플,30ml,"33,000원",258위,안티에이징에센스,...,"['피부컨디셔닝제', '변성제, 착향제, 헤어컨디셔닝제, 보습제, 피부보호제, 피부...",0개,[],[],[],0개,[],[],[],"['스킨케어', '에센스/세럼', '안티에이징에센스']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64238,,세븐피엠,NaN,https://www.glowpick.com/products/104861,104861,포멀 포마드,80ml,"15,000원",57위,남성 헤어스타일링,...,[],0개,[],[],[],0개,[],[],[],"['남성화장품', '남성 헤어', '남성 헤어스타일링']"
64239,,세븐피엠,NaN,https://www.glowpick.com/products/116353,116353,마스터 커버스틱,1g,"12,900원",[],[],...,[],0개,[],[],[],0개,[],[],[],"['남성화장품', '남성 메이크업', '남성 페이스메이크업']"
64240,,세븐피엠,NaN,https://www.glowpick.com/products/113203,113203,제로 왁스,80ml,"15,500원",[],[],...,[],0개,[],[],[],0개,[],[],[],"['남성화장품', '남성 헤어', '남성 헤어스타일링']"
64241,,세븐피엠,NaN,https://www.glowpick.com/products/112924,112924,리얼콤 다운펌,100ml,"13,900원",[],[],...,[],0개,[],[],[],0개,[],[],[],"['헤어', '헤어메이크업', '염색/탈색/펌']"


In [508]:
df_prd_info = pd.DataFrame(columns = ['브랜드 link','브랜드명','브랜드 id','제품 link','제품 id','제품명','제품 용량','제품 가격','유사 제품군 랭킹(int)',
                                         '유사 제품군 랭킹(text)','브랜드 내 랭킹(int)','제품 수상명','제품 수상 부문','제품 수상 순위','제품 설명','제품 컬러/타입',
                                         '제품 판매처','제품 성분구성: 위험도 미정 (갯수)','제품 성분구성: 위험도 미정 (성분(한글))',
                                         '제품 성분구성: 위험도 미정 (성분(영어))','제품 성분구성: 위험도 미정 (성분(설명))',
                                         '제품 성분구성: 낮은 위험도 (갯수)','제품 성분구성: 낮은 위험도 (성분(한글))',
                                         '제품 성분구성: 낮은 위험도 (성분(영어))','제품 성분구성: 낮은 위험도 (성분(설명))',
                                         '제품 성분구성: 중간 위험도 (갯수)','제품 성분구성: 중간 위험도 (성분(한글))',
                                         '제품 성분구성: 중간 위험도(성분(영어))','제품 성분구성: 중간 위험도 (성분(설명))',
                                         '제품 성분구성: 높은 위험도 (갯수)','제품 성분구성: 높은 위험도 (성분(한글))',
                                         '제품 성분구성: 높은 위험도 (성분(영어))','제품 성분구성: 높은 위험도 (성분(설명))',
                                         '제품 카테고리','리뷰','별점','리뷰 일자',
                                         '사용자 리뷰 개수','사용자 피부 타입','사용자 나이','사용자 성별','사용자 이름', 'image src'])

In [519]:
need_to_crawl_lst[48]

'https://www.glowpick.com/products/150329'

In [515]:
exception = []
i = 22

for prd_link in tqdm(need_to_crawl_lst[22:]):
    prd_info = scrap_product_details(prd_link)
    df_prd_info.loc[i] = prd_info
    i += 1

  0%|          | 0/33 [00:00<?, ?it/s]



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.90 Safari/537.36


...

Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 6.1; rv:14.0) Gecko/20100101 Firefox/18.0.1


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [524]:
prd_info = scrap_product_details(need_to_crawl_lst[48])



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [525]:
prd_info

{'브랜드 link': nan,
 '브랜드명': '비바이바닐라',
 '브랜드 id': nan,
 '제품 link': 'https://www.glowpick.com/products/150329',
 '제품 id': '150329',
 '제품명': '컨투어 브러쉬',
 '제품 용량': '1ea',
 '제품 가격': '14,000원',
 '유사 제품군 랭킹(int)': '190위',
 '유사 제품군 랭킹(text)': '치크브러시',
 '브랜드 내 랭킹(int)': '39위',
 '제품 수상명': [],
 '제품 수상 부문': [],
 '제품 수상 순위': [],
 '제품 설명': '* 본 제품은 리뉴얼된 제품입니다.뭉침 없는 블렌딩으로 자연스러운 컨투어링을 표현하는 비건 브러쉬',
 '제품 컬러/타입': '',
 '제품 판매처': [],
 '제품 성분구성: 위험도 미정 (갯수)': '0개',
 '제품 성분구성: 위험도 미정 (성분(한글))': [],
 '제품 성분구성: 위험도 미정 (성분(영어))': [],
 '제품 성분구성: 위험도 미정 (성분(설명))': [],
 '제품 성분구성: 낮은 위험도 (갯수)': '0개',
 '제품 성분구성: 낮은 위험도 (성분(한글))': [],
 '제품 성분구성: 낮은 위험도 (성분(영어))': [],
 '제품 성분구성: 낮은 위험도 (성분(설명))': [],
 '제품 성분구성: 중간 위험도 (갯수)': '0개',
 '제품 성분구성: 중간 위험도 (성분(한글))': [],
 '제품 성분구성: 중간 위험도(성분(영어))': [],
 '제품 성분구성: 중간 위험도 (성분(설명))': [],
 '제품 성분구성: 높은 위험도 (갯수)': '0개',
 '제품 성분구성: 높은 위험도 (성분(한글))': [],
 '제품 성분구성: 높은 위험도 (성분(영어))': [],
 '제품 성분구성: 높은 위험도 (성분(설명))': [],
 '제품 카테고리': ['뷰티툴', '브러시', '치크브러시'],
 '리뷰': ['우연한 기회로 접하여 이제서야 써봤

In [526]:
df_prd_info.loc[48] = prd_info

In [527]:
df_prd_info

,브랜드 link,브랜드명,브랜드 id,제품 link,제품 id,제품명,제품 용량,제품 가격,유사 제품군 랭킹(int),유사 제품군 랭킹(text),...,제품 카테고리,리뷰,별점,리뷰 일자,사용자 리뷰 개수,사용자 피부 타입,사용자 나이,사용자 성별,사용자 이름,image src
0,NaN,닐스야드 레머디스,NaN,https://www.glowpick.com/products/150283,150283,오렌지플라워 페이셜 워시,100ml,"30,800원",,,...,"[클렌징, 페이셜클렌저, 클렌징폼]",[],[],[],[],[],[],[],[],NaN
1,NaN,닐스야드 레머디스,NaN,https://www.glowpick.com/products/150287,150287,팔마로사 페이셜 워시,100ml,"30,800원",,,...,"[클렌징, 페이셜클렌저, 클렌징폼]",[],[],[],[],[],[],[],[],NaN
2,NaN,닥터레미듀,NaN,https://www.glowpick.com/products/150292,150292,에이씨 포어 스케일링 마이크로 버블 클렌징 폼,200ml,"20,000원",,,...,"[클렌징, 페이셜클렌저, 클렌징폼]",[],[],[],[],[],[],[],[],NaN
3,NaN,닥터헤디슨,NaN,https://www.glowpick.com/products/150451,150451,버블리 포밍 딥 클렌저,250ml,"60,000원",646위,클렌징폼,...,"[클렌징, 페이셜클렌저, 클렌징폼]",[딥 클렌저 제품임에도 세안 후 당기는 느낌이 전혀 없고 촉촉해서 좋았습니다 간단한...,[4],[8일전],[리뷰 27],[복합성],[28세],[여],[ppoopoo],NaN
4,NaN,더샘,NaN,https://www.glowpick.com/products/150494,150494,힐링 티 가든 화이트티 클렌징 워터,300ml,"5,500원",241위,클렌징워터,...,"[클렌징, 메이크업클렌저, 클렌징워터]",[클렌징 워터라고 해서 솜에 듬뿍 묻혀서 지웠는데 하나도... 안 지워져요. 전 이...,[1],[26일전],[리뷰 53],[복합성],[22세],[여],[ma-yeon.00],NaN
5,NaN,더샘,NaN,https://www.glowpick.com/products/150624,150624,어반 에코 하라케케 크림,50ml,"17,000원",,,...,"[스킨케어, 크림, 보습크림]",[],[],[],[],[],[],[],[],NaN
6,NaN,더샘,NaN,https://www.glowpick.com/products/150511,150511,힐링 티 가든 그린티 클렌징 티슈,60ea,"5,500원",,,...,"[클렌징, 메이크업클렌저, 클렌징티슈]",[],[],[],[],[],[],[],[],NaN
7,NaN,더샘,NaN,https://www.glowpick.com/products/150442,150442,씨앤소 에이씨 컨트롤 딥 클렌징 폼,120ml,"9,000원",,,...,"[클렌징, 페이셜클렌저, 클렌징폼]",[],[],[],[],[],[],[],[],NaN
8,NaN,데저트에센스,NaN,https://www.glowpick.com/products/150927,150927,코코넛 샴푸,237ml,"15,900원",,,...,"[헤어, 샴푸, 보습/영양샴푸]",[],[],[],[],[],[],[],[],NaN
9,NaN,데저트에센스,NaN,https://www.glowpick.com/products/150928,150928,코코넛 컨디셔너,237ml,"15,900원",,,...,"[헤어, 린스/컨디셔너, 보습/영양린스]",[],[],[],[],[],[],[],[],NaN


In [530]:
v4_update = pd.read_csv("v4_update.csv")
v4_update = v4_update.iloc[:, 1:]

In [531]:
v4_update = pd.concat([v4_update, df_prd_info])

In [534]:
v4_update = v4_update.reset_index()

In [535]:
v4_update.to_csv("v4_update.csv")

In [506]:
def scrap_product_details(prd_link):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    #options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    wd.get(prd_link)
    time.sleep(5)
    wd.maximize_window()
    
    socket.setdefaulttimeout(30)
        
    try:
        
        wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
        time.sleep(5)

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        
        time.sleep(3)
        
        #img scr
        img_src = wd.find_element_by_xpath("/html/body/div/div/div/div/main/div/section/div[1]/div/div/img").get_attribute('src')
        img_src = re.sub("w=[0-9]*", "w=500", img_src)
        
        time.sleep(3)
        brd_name = wd.find_element_by_class_name("product__summary__brand__name").text
        brd_id = np.nan
        brd_link = np.nan

        prd_name = wd.find_element_by_class_name("product__summary__name").text
        prd_id = prd_link.split('/')[-1]

        prd_volume = wd.find_element_by_class_name("offer__volume-price").text.split(' / ')[0]
        prd_price = wd.find_element_by_class_name("offer__volume-price").text.split(' / ')[1]

        time.sleep(2)

        element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[3]')
        wd.execute_script("arguments[0].scrollIntoView(true);", element)

        time.sleep(2)
        info_article = bs.select(".info__article")


        #랭킹
        if '랭킹' in str(bs.select(".info__article")):
            prd_grp_text = wd.find_element_by_class_name("rank-item__name").text #유사제품군랭킹(text)
            prd_grp_rank = wd.find_elements_by_class_name("rank-item__rank")[0].text #유사제품군랭킹(text)
            prd_brd_rank = wd.find_elements_by_class_name("rank-item__rank")[1].text #브랜드 내 링킹(int)

            if '수상' in str(bs.select(".info__article")):
                view_all_award_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[3]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[4]/h3/button'
            else:
                view_all_award_xpath = ''
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[3]/h3/button'


        else: #랭킹이 없는 경우

            prd_grp_text = ''
            prd_grp_rank = ''
            prd_brd_rank = ''

            if '수상' in str(bs.select(".info__article")):
                view_all_award_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[1]/h3/button'
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[3]/h3/button'

            else:
                view_all_award_xpath = ''
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[1]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'


        #수상내역 전체보기
        award_name = [] #제품 수상명
        award_part = [] #제품 수상 부문
        award_rank = [] #제품 수상 순위

        if view_all_award_xpath != '':
            wd.find_element_by_xpath(view_all_award_xpath).click()
            html = wd.page_source
            bs = BeautifulSoup(html, 'html.parser')
            time.sleep(5)

            for i in range(len(bs.select('.awards__item__text__name'))):
                award_name.append(bs.select('.awards__item__text__name')[i].text.strip())
                award_part.append(bs.select('.awards__item__text__award')[i].text.strip())
                award_rank.append(bs.select(".awards__item__text__rank")[i].text.strip())
        #     award_name = re.findall('<p class="awards__item__text__name" (.*?)id="(.*?)">', bs.select('.awards__item__text__name')) #제품 수상명
        #     #re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0])
        #     award_part = bs.select('.awards__item__text__award') #제품 수상 부문
        #     award_rank = bs.select(".awards__item__text__rank") #제품 수상 순위
            wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/h1/button/span').click()
        time.sleep(2)


        #성분 구성 전체보기
        wd.find_element_by_xpath(view_all_ingredient_xpath).click()
        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(5)

        ingredient_unknown_cnt = wd.find_elements_by_class_name("tag__bg__label")[0].text #위험도 미정 개수
        ingredient_low_cnt = wd.find_elements_by_class_name("tag__bg__label")[1].text #낮은 위험도 개수
        ingredient_mid_cnt = wd.find_elements_by_class_name("tag__bg__label")[2].text #중간 위험도 개수
        ingredient_high_cnt = wd.find_elements_by_class_name("tag__bg__label")[3].text #높은 위험도 개수

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(3)

        ingredient_lst_unknown = []
        ingredient_lst_low = []
        ingredient_lst_mid = []
        ingredient_lst_high = []

        ingredient_lst = bs.select('.ingredient__list__item.item')

        for index,li in enumerate(ingredient_lst):
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--unknown"}):
                ingredient_lst_unknown.append(li)
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--low"}):
                ingredient_lst_low.append(li)
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--mid"}):
                ingredient_lst_mid.append(li)
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--high"}):
                ingredient_lst_high.append(li)

        ingredient_unknown_kor_lst = []
        ingredient_unknown_eng_lst = []
        ingredient_unknown_desc_lst = []

        for ingredient_unknown in ingredient_lst_unknown:

            if ingredient_unknown.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_unknown_kor = ingredient_unknown.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_unknown_kor = ingredient_unknown_kor.strip()
                ingredient_unknown_kor_lst.append(ingredient_unknown_kor)

            if ingredient_unknown.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_unknown_eng = ingredient_unknown.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_unknown_eng = ingredient_unknown_eng.strip()
                ingredient_unknown_eng_lst.append(ingredient_unknown_eng)


            if ingredient_unknown.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_unknown_desc = ingredient_unknown.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_unknown_desc = ingredient_unknown_desc.strip()
                ingredient_unknown_desc_lst.append(ingredient_unknown_desc)

        ingredient_low_kor_lst = []
        ingredient_low_eng_lst = []
        ingredient_low_desc_lst = []

        for ingredient_low in ingredient_lst_low:

            if ingredient_low.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_low_kor = ingredient_low.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_low_kor = ingredient_low_kor.strip()
                ingredient_low_kor_lst.append(ingredient_low_kor)

            if ingredient_low.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_low_eng = ingredient_low.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_low_eng = ingredient_low_eng.strip()
                ingredient_low_eng_lst.append(ingredient_low_eng)


            if ingredient_low.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_low_desc = ingredient_low.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_low_desc = ingredient_low_desc.strip()
                ingredient_low_desc_lst.append(ingredient_low_desc)

        ingredient_mid_kor_lst = []
        ingredient_mid_eng_lst = []
        ingredient_mid_desc_lst = []

        for ingredient_mid in ingredient_lst_mid:

            if ingredient_mid.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_mid_kor = ingredient_mid.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_mid_kor = ingredient_mid_kor.strip()
                ingredient_mid_kor_lst.append(ingredient_mid_kor)

            if ingredient_mid.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_mid_eng = ingredient_mid.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_mid_eng = ingredient_mid_eng.strip()
                ingredient_mid_eng_lst.append(ingredient_mid_eng)


            if ingredient_mid.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_mid_desc = ingredient_mid.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_mid_desc = ingredient_mid_desc.strip()
                ingredient_mid_desc_lst.append(ingredient_mid_desc)


        ingredient_high_kor_lst = []
        ingredient_high_eng_lst = []
        ingredient_high_desc_lst = []

        for ingredient_high in ingredient_lst_high:

            if ingredient_high.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_high_kor = ingredient_high.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_high_kor = ingredient_high_kor.strip()
                ingredient_high_kor_lst.append(ingredient_high_kor)

            if ingredient_high.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_high_eng = ingredient_high.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_high_eng = ingredient_high_eng.strip()
                ingredient_high_eng_lst.append(ingredient_high_eng)


            if ingredient_high.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_high_desc = ingredient_high.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_high_desc = ingredient_high_desc.strip()
                ingredient_high_desc_lst.append(ingredient_high_desc)

        wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/h1/button/span').click()
        time.sleep(2)


        #제품 설명 전체보기
        wd.find_element_by_xpath(view_all_prd_desc_xpath).click()

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(5)

        ##제품 설명
        try:
            prd_desc = wd.find_element_by_class_name('descriptions__article__pre').text.replace("\n", '')
            time.sleep(2)
        except:
            prd_desc = np.nan

        ##카테고리
        category = []
        try:
            category.append(wd.find_element_by_class_name("descriptions__article__category").text)
        except:
            pass
        
        try:
            category_link = bs.select('.descriptions__article__category.descriptions__article__category-link')#제품 카테고리(하위)
            category_link = re.findall(">(.*?)</span>", str(category_link))
        except:
            category_link = []
            
        category = category + category_link

        ##컬러/타입
        try:
            color_type = wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/div/article[2]/div').text
            time.sleep(3)
        except:
            color_type=''
            pass

        ##판매처
        try:
            store = str(bs.select('.stores__store__name')).replace(' ', '')
            store = re.findall("\n(.*?)\n", store)
            time.sleep(3)
        except:
            store = []

        wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/h1/button/span').click()
        time.sleep(2)

        #리뷰
        element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/section')
        wd.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(5)

        review = []
        rating = []
        review_date = []
        user_review_cnt = []
        user_skin_type = []
        user_age = []
        user_sex = []
        user_name = []

        review_len = len(wd.find_elements_by_class_name("cutter__pre"))
        info_details = bs.select(".info__details")
        time.sleep(5)

        for i in range(review_len):
            review.append(wd.find_elements_by_class_name("cutter__pre")[i].text.replace('\n', ''))
            review_date.append(bs.select(".review__side-info__created-at")[i].text.strip())
            user_age.append(str(info_details[i]).split('\n')[1].strip())
            user_skin_type.append(str(info_details[i]).split('\n')[3].strip())
            user_sex.append(str(info_details[i]).split('\n')[5].strip())
            user_review_cnt.append(str(info_details[i]).split('\n')[7].strip())
            user_name.append(re.findall('<p class="info__details__nickname" data(.*?)>(.*?)</p>', str(info_details[i]).split('\n')[0])[0][-1])
        time.sleep(3)
        
        star = bs.select(".stars")
        for i in range(len(star)-2):
            rating.append(str(star[i+2]).split('\n')[-2].strip())

        wd.close()

        prd_info = {
            "브랜드 link": brd_link,
            "브랜드명": brd_name,
            "브랜드 id": brd_id,
            "제품 link": prd_link,
            "제품 id": prd_id,
            "제품명": prd_name,
            "제품 용량": prd_volume,
            "제품 가격": prd_price,
            "유사 제품군 랭킹(int)": prd_grp_rank,
            "유사 제품군 랭킹(text)": prd_grp_text,
            "브랜드 내 랭킹(int)": prd_brd_rank,
            "제품 수상명": award_name,
            "제품 수상 부문": award_part,
            "제품 수상 순위": award_rank,
            "제품 설명": prd_desc,
            "제품 컬러/타입": color_type,
            "제품 판매처": store,
            "제품 성분구성: 위험도 미정 (갯수)": ingredient_unknown_cnt,
            "제품 성분구성: 위험도 미정 (성분(한글))": ingredient_unknown_kor_lst,
            "제품 성분구성: 위험도 미정 (성분(영어))": ingredient_unknown_eng_lst,
            "제품 성분구성: 위험도 미정 (성분(설명))": ingredient_unknown_desc_lst,
            "제품 성분구성: 낮은 위험도 (갯수)": ingredient_low_cnt,
            "제품 성분구성: 낮은 위험도 (성분(한글))": ingredient_low_kor_lst,
            "제품 성분구성: 낮은 위험도 (성분(영어))": ingredient_low_eng_lst,
            "제품 성분구성: 낮은 위험도 (성분(설명))": ingredient_low_desc_lst,
            "제품 성분구성: 중간 위험도 (갯수)": ingredient_mid_cnt,
            "제품 성분구성: 중간 위험도 (성분(한글))": ingredient_mid_kor_lst,
            "제품 성분구성: 중간 위험도(성분(영어))": ingredient_mid_eng_lst,
            "제품 성분구성: 중간 위험도 (성분(설명))": ingredient_mid_desc_lst,
            "제품 성분구성: 높은 위험도 (갯수)": ingredient_high_cnt,
            "제품 성분구성: 높은 위험도 (성분(한글))": ingredient_high_kor_lst,
            "제품 성분구성: 높은 위험도 (성분(영어))": ingredient_high_eng_lst,
            "제품 성분구성: 높은 위험도 (성분(설명))": ingredient_high_desc_lst,
            "제품 카테고리": category,
            "리뷰": review,
            "별점": rating,
            "리뷰 일자": review_date,
            "사용자 리뷰 개수": user_review_cnt,
            "사용자 피부 타입": user_skin_type,
            "사용자 나이": user_age,
            "사용자 성별": user_sex,
            "사용자 이름": user_name,
            "이미지 src": img_src
        }
        
        return prd_info
        
    except Exception as e:
        exception.append(prd_link)
        print(e)